<a href="https://colab.research.google.com/github/Hernan4444/Hernan4444-diplomado-sistemas-recomendadores-2020-1/blob/master/Diplomado_Alumno_2020_Sistemas_Recomendadores_Pyreclab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica Pyreclab 1

En este práctico vamos a utilizar la biblioteca de Python [pyreclab](https://github.com/gasevi/pyreclab) desarrollado por los Laboratorios IALab y SocVis de la Pontificia Universidad Católica de Chile, para aprender sobre algoritmos básicos y tradicionales de sistemas de recomendacion:


*   Most Popular
*   Item Average Rating
*   User KNN (Filtrado colaborativo basado en usuarios)
*   Item KNN (Filtrado colaborativo basado en items)
*   FunkSVD (Factorización matricial SVD regularizada)


**Autores**: Denis Parra, Gabriel Sepúlveda

**Ayudantes** Manuel Cartagena y Hernán Valdivieso


**Trabajo de a pares**

**Nombre 1**: tu nombre aqui

**Nombre 2**: nombre de tu compañero aquí.

# Índice

>[Practica Pyreclab 1](#scrollTo=NC-ceGb8LRLT)

>[Índice](#scrollTo=2j02H66f87eV)

>[Descargando la información](#scrollTo=9qKTQ2V5VKSX)

>[Revisar archivos descargados](#scrollTo=liJ3L2kl4g1o)

>>[Como ver la información de una o más películas](#scrollTo=jqXxj773Foo8)

>>[Preparar entorno](#scrollTo=P3K5VPZoGSYX)

>[Probar Pyreclab](#scrollTo=tGbAZ8yAMAk5)

>>[Most Popular](#scrollTo=tGbAZ8yAMAk5)

>>[Item Avg](#scrollTo=d72jgXBsML5a)

>>>[Actividad 1:](#scrollTo=tMiP_Z2VYZB9)

>>>[Actividad 2:](#scrollTo=FAmYXWtWXg3D)

>>>[Actividad 3](#scrollTo=_ZdUT93BMYgq)

>>>[Actividad 4](#scrollTo=4WV4L4lmOW5-)

>>[UserKNN](#scrollTo=2e0ZeZXzMhU3)

>>>[Actividad 5](#scrollTo=I-iqQvAdTTCP)

>>>[Actividad 6](#scrollTo=Wp-2JLSBJhPv)

>>[Evaluar en base a error de prediccion (MAE y RMSE) UserKnn](#scrollTo=6_Kp3zRwJ9gv)

>>>[Actividad 7](#scrollTo=jrjMzM5zioaP)

>>[Actividad 8 ItemKNN](#scrollTo=FqgkF0dNMo9c)

>>[Actividad 9 SlopeOne](#scrollTo=NAE5IZzXMqiJ)



# Descargando la información

Vaya ejecutando cada celda presionando el botón de **Play** o presionando Ctrl+Enter (Linux y Windows) o Command+Enter (Macosx) para descargar las bases de datos

In [0]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

In [0]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

In [0]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

# Revisar archivos descargados

Revisemos cómo es uno de estos archivos:

In [0]:
import pandas as pd

train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)

train_file.head()

Por otra parte, para obtener información adicional de cada película tal como *título*, *fecha de lanzamient*o, *género*, etc., cargaremos el archivo de items descargado ( *u.item* ) para poder mapear cada identificador de ítem al conjunto de datos que lo describe.
Revisemos el contenido de este archivo

In [0]:
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

pd.options.display.max_columns = None

info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

info_file.head()

El comando `pd.options.display.max_columns = None` permite que la tabla se pueda mostrar completamente de forma horizontal, es decir, que muestre todas sus columnas.

## Como ver la información de una o más películas
Para esto se hace 2 pasos, 
1. Determinar los índices de las películas
2. Pedir a `info_file` las columnas
3. (adicional) Indicar alguna columna en específico

In [0]:
# ejemplo de cómo visualizar titulos de peliculas en base a sus IDs
pelis = [5,4,1]
info_file.loc[pelis]

Para éste código:
1. `pelis = [5,4,1]` indica que queremos las películas cuyo índice son 5, 4 y 1
2. `info_file.loc[pelis]` el método `loc` permite acceder a esas columna
A continuación se verá como obtener una columna en específico. Para esto solo es necesario agregar corchetes y el nombre de la columna

In [0]:
info_file.loc[pelis]['title']

## Preparar entorno
Primero es necesario instalar una librería para realizar recomendaciones. Esta se llama ***pyreclab***

In [0]:
!pip install pyreclab

Luego necesitamos importar pyreclab y numpy, 2 librerías a utilizar en este práctico

In [0]:
import pyreclab
import numpy as np

# Probar Pyreclab

## Most Popular

Probar método de recomendación basado en ***Most Popular***

In [0]:
# declarar el objeto recomendador de la clase MostPopular
mymp = pyreclab.MostPopular(dataset = 'u1.base',dlmchar = b'\t',header = False,usercol = 0,itemcol = 1, ratingcol = 2 )

In [0]:
# entrenamos el modelo con los datos existentes

# el método calcula el numero de veces que fue consumido cada item, 
# así, puede luego recomendar los mas populares
mymp.train()

In [0]:
# con este método no podemos predecir ratings, pero sí podemos recomendar
# la recomendación es no personalizada (igual recomendación a todos los usuarios)
reclist = mymp.recommend("457")
print(reclist)

`reclist` es una lista de 10 índices que representan las películas a recomendar. A continuación se muestra como transformar esos índicas a números (`int`) y luego usar el comando `loc` para obtener la información de las películas, en partícular los títulos.

In [0]:
# con esto podemos ver las recomendaciones con titulo de película
pelis = np.array(reclist).astype(int)
info_file.loc[pelis]['title']

## Item Avg

Probar método de recomendación basado en ***ItemAvg***

In [0]:
# declarar el objeto recomendador ItemAvg
myitemavg = pyreclab.ItemAvg(dataset = 'u1.base',dlmchar = b'\t',header = False,usercol = 0,itemcol = 1, ratingcol = 2 )


In [0]:
# entrenamos el modelo con los datos existentes
myitemavg.train()

### Actividad 1:
Responder la pregunta: ¿Qué cree usted que hace la función `train()` del método de recomendación `ItemAvg()`` ?

Respuesta:

In [0]:
# Predecir rating que el usuario ID 457 le dará al ítem ID 37
myitemavg.predict("457","37")

### Actividad 2:

Ejecute el mismo comando anterior para predecir esta vez el rating que el usuario 498 dará al item 37 ¿Qué rating da? ¿Cómo se explica este resultado? (Asegurese de generar tanto el código como responder ambas preguntas)

Respuesta:

In [0]:
# Generar lista de 5 recomendaciones para el usuario con ID 457

reclist_iavg = myitemavg.recommend("457",5)
print(reclist_iavg)

### Actividad 3

Genere una lista de 11 recomendaciones para el usario ID 478 ¿Cómo se explican las recomendaciones del método para este usuario comparadas con las del usuario 457?

In [0]:
# genera aqui las recomendacion para el usuario 478



Respuesta: 

### Actividad 4
¿Cuáles son los títulos de las películas recomendadas por el método **ItemAvg** para el usuario 457 ? ¿Qué le parecen estas recomendaciones comparadas con el método **MostPopular**?

In [0]:
# (pista) recuerde los comandos usados en el método anterior


## UserKNN

Probar método de recomendación basado en ***UserKNN***

In [0]:
# declarar el objeto recomendador UserKnn
myUserKnn = pyreclab.UserKnn(dataset = 'u1.base',dlmchar = b'\t',header = False,usercol = 0,itemcol = 1, ratingcol = 2 )


In [0]:
# entrenamos el modelo con los datos existentes
# recuerde que en este metodo puede probar el parámetro k de cantidad de vecinos
# así como la métrica de similaridad (pearson, cosine)
myUserKnn.train(7,'pearson')

### Actividad 5

Según el modelo de recomendación UserKnn, qué rating le dará el usuario 457 al item 37 ? Para esto ocupe el método `predict` tal como lo hizo en ***ItemAVG***.



Respuesta: 

### Actividad 6

Generar lista de 6 recomendaciones para el usuario con ID 457. Indique qué películas son con todas sus columnas. (Recuerde el comando `loc` vista anteriormete)

In [0]:
# (pista) recuerde los comandos usados en el método anterior


Escriba los nombres de las películas recomendadas por el método knn

Respuesta: 

## Evaluar en base a error de prediccion (MAE y RMSE) ***UserKnn***

In [0]:
k_values = [5, 10, 30, 50] # definir distintos valores de vecinos (k)
mae_values = [] # para almacenar resultados de MAE
rmse_values = [] # para almacenar valores de RMSE

for k in k_values:
  
  myUserKnn.train(k,'cosine')
  predlist, mae, rmse = myUserKnn.test( input_file = 'u1.test',dlmchar = b'\t', header = False, usercol = 0, itemcol = 1, ratingcol = 2, output_file = 'predictions.csv' )
  rmse_values.append(rmse)   

print(rmse_values)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(k_values, rmse_values, 'r-')
plt.plot(k_values, rmse_values, 'bo')
plt.show()


### Actividad 7
Según los resultados: ¿ qué numero de vecinos es mejor para hacer recomendaciones con un menor error de predicción ?

## Actividad 8 ItemKNN

Probar método de recomendación basado en ***ItemKNN***. **Replicar** todo el análisis de UserKNN con el método ItemKNN. En particular:
1. Entrene el modelo
2. Prediga el rating que le dará un usuario a un ítem. El usuario e ítem quedan a tu criterio
3. Genera una lista de 5 recomendaciones para un usuario e indique qué películas son


In [0]:
# Puede agregar las celdas de código y de texto que estimes conveniente.

## Actividad 9 SlopeOne

Probar método de recomendación basado en ***SlopeOne***. **Replicar** todo el análisis de UserKNN con el método SlopeOne. En particular:
1. Entrene el modelo
2. Prediga el rating que le dará un usuario a un ítem. El usuario e ítem quedan a tu criterio
3. Genera una lista de 5 recomendaciones para un usuario e indique qué películas son


In [0]:
# Puede agregar las celdas de código y de texto que estimes conveniente.